# SQL Select

**Wymagania**

- założona baza Northwind (skrypt dostępny na kanale lub na [GitHub](https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/northwind-pubs))

In [ ]:
USE Northwind;
GO

## Zadanie 1

_Wyświetlenie danych wszystkich zamówień_


Nowe słowa kluczowe:

- __FROM__ [db].[schema].table_name - tabela z której pobieramy dane
- __SELECT__ - lista kolumn w wyniku, __*__ (wildcard) - zwraca wszystkie kolumny występujące w zapytaniu

In [ ]:
SELECT * 
FROM Orders;

## Zadanie 2

_Wyświetlenie danych wszystkich zamówień dostarczonych do Meksyku (Mexico), Niemiec (Germany) lub Brazylii (Brazil)_


Nowe słowa kluczowe:

- __WHERE__ - umożliwia filtrowanie rekordów
- __OR__ / __AND__ - operatory logiczne
- __IN__ - sprawdzanie przynależności wartości w kolumnie do kolekcji.

IN nie powinno być stosowane gdy podana kolekcja jest duża (np, kolekcja nie powinna być wynikiem podzapytania). W takim przypadku lepiej wykorzystać JOIN

In [ ]:
SELECT * 
FROM Orders 
WHERE ShipCountry = 'Brazil' OR ShipCountry = 'Mexico' OR ShipCountry = 'Germany'

In [ ]:
SELECT * 
FROM Orders 
WHERE ShipCountry IN ('Brazil','Mexico','Germany')

## Zadanie 3

_Wyświetlenie nazw miast w Niemczech, do których dostarczono 
produkty_


Nowe słowa kluczowe:

- __DISTINCT__ - umożliwia usunięcie duplikatów __rekordów__ z wyniku

In [ ]:
SELECT DISTINCT ShipCity 
FROM Orders
WHERE ShipCountry = 'Germany'

## Zadanie 4

_Wyświetlenie danych zamówień złożonych w lipcu 1996_


Filtrowanie dat możemy zrealizować na dwa sposoby:

1. przy pomocy porównania daty z ograniczeniem dolnym oraz górnym
2. przez zastosowanie funkcji zwracających część daty (np. miesiąc, rok)

Rozwiązanie nr. 1 jest lepsze ponieważ umożliwia wykorzystanie indeksów co umożliwia ograczenie liczby odczytanych rekordów. W drugim przypadku funkcje muszą być obliczone dla każdego rekordu.

O formatach dat:

https://docs.microsoft.com/en-us/sql/t-sql/data-types/date-transact-sql?view=sql-server-ver15

In [ ]:
SELECT * 
FROM Orders 
WHERE OrderDate >= '1996-07-01' AND OrderDate < '1996-08-01'

In [ ]:
SELECT * 
FROM Orders 
WHERE MONTH(OrderDate) = 7 AND YEAR(OrderDate) = 1996

## Zadanie 5

_Wyświetlenie pierwszych 10 znaków nazw firm, po konwersji 
do dużych znaków_


Nowe słowa kluczowe:

- __AS__ - nadawanie aliasu dla kolumny / tabeli - podawanie AS jest opcjonalne

O funkcjach dostępnych w ramach T-SQL

https://docs.microsoft.com/en-us/sql/t-sql/functions/functions?view=sql-server-ver15

In [ ]:
SELECT UPPER(SUBSTRING(CompanyName, 1, 10)) AS 'Company code' 
FROM Customers

In [ ]:
SELECT UPPER(LEFT(CompanyName, 10)) 'Company code' 
FROM Customers

## Zadanie 6

_Wyświetlenie danych wszystkich zamówień złożonych przez 
klientów z Francji_


Dzięki __JOIN__ możemy dołączać nowe kolumny z innej tabeli. Logicznie operację możemy interpretować jako
iloczyn kartezjański zbiorów rekordów z tabeli przed i po JOIN. Następnie klauzula __ON__ określa warunek,
według którego filtrowane są wynikowe rekordy. Najczęściej w tym miejscu wykorzystywane są kolumny występujące w relacjach, ale nic nie stoii na przeszkodzie, żeby był to dowolny warunek logiczny.

Poniższe zapytania są równoważne.

Gdy w zapytaniu mamy do czynienia z kilkoma tabelami dobrą praktyką jest odwoływanie się do poszczególnych kolumn z wykorzystaniem nazw tabel. Jeszcze częstszą praktyką jest nadawanie aliasów tabelom w celu skrócenia zapisu. 

Podanie wielu tabel w klazuli FROM jest toższame z CROSS JOIN.

JOIN: https://docs.microsoft.com/en-us/sql/relational-databases/performance/joins?view=sql-server-ver15

In [ ]:
SELECT Orders.* 
FROM Orders
JOIN Customers ON Orders.CustomerID = Customers.CustomerID
WHERE Customers.Country = 'France'

In [ ]:
SELECT O.* 
FROM Orders O
JOIN Customers C ON O.CustomerID = C.CustomerID
WHERE C.Country = 'France'

In [ ]:
SELECT O.* 
FROM Orders O
JOIN Customers C ON 1 = 1
WHERE O.CustomerID = C.CustomerID 
    AND C.Country = 'France'

In [ ]:
SELECT O.* 
FROM Orders O, Customers C
WHERE O.CustomerID = C.CustomerID 
    AND C.Country = 'France'

## Zadanie 7

_Wyświetlenie wszystkich krajów dostawy dla zamówień 
złożonych przez klientów z Niemiec_

In [ ]:
SELECT DISTINCT o.ShipCountry 
FROM Orders o 
JOIN Customers c ON o.CustomerID = c.CustomerID 
WHERE c.Country = 'Germany'

## Zadanie 8

_Znalezienie wszystkich zamówień dostarczonych do innego 
kraju niż kraj, z którego pochodził klient_

In [ ]:
SELECT o.* 
FROM Orders o 
JOIN Customers c ON o.CustomerID = c.CustomerID 
WHERE c.Country != o.ShipCountry

## Zadanie 9

_Znalezienie wszystkich klientów, którzy nigdy nie złożyli 
żadnych zamówień_


Czasami chcemy sprawdzić czy podzapytanie zwraca __niepusty__ wynik. Służy do tego słowo kluczowe __EXISTS__.

Powyższe zadanie możemy zapisać w lekko zmienionej formie.

```
Dla każdego klienta sprawdź czy składał jakiekolwiek zamówienia
```

Zwróćmy uwagę na warunek WHERE występujący w podzapytaniu - jest on tożsamy z tym co musielibyśmy zapisać po JOIN.



O EXISTS 
  https://docs.microsoft.com/en-us/sql/t-sql/language-elements/exists-transact-sql?view=sql-server-ver15


In [ ]:
SELECT * 
FROM Customers c 
WHERE NOT EXISTS (SELECT * FROM Orders o WHERE o.CustomerID = c.CustomerID)

Poniższy fragment jest często nazywany jako LEFT ANTI-JOIN, ponieważ w wyniku pojawią się rekordy z tabeli lewej, które __NIE__ występują w tabeli prawej.

```
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
WHERE o.CustomerID IS NULL;
```


In [ ]:
SELECT * 
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
WHERE o.CustomerID IS NULL;

## Zadanie 10

_Znalezienie wszystkich klientów, którzy nigdy nie zamówili 
produktu Chocolade_

In [ ]:
SELECT * 
FROM Customers c 
WHERE NOT EXISTS (
  SELECT *
  FROM Orders o 
  WHERE o.CustomerID = c.CustomerID
    AND EXISTS (
      SELECT * 
      FROM [Order Details] od 
      JOIN Products p ON p.ProductID = od.ProductID 
      WHERE od.OrderID = o.OrderID AND p.ProductName = 'Chocolade'
      )
  )


In [ ]:
SELECT * 
FROM Customers c 
WHERE NOT EXISTS (
  SELECT *
  FROM Orders o 
  WHERE o.CustomerID = c.CustomerID 
    AND EXISTS (
      SELECT * 
      FROM [Order Details] od 
      WHERE od.OrderID = o.OrderID 
        AND od.ProductID = (
          SELECT ProductID 
          FROM Products p 
          WHERE p.ProductName = 'Chocolade'
        )
      )
  )


In [ ]:
SELECT * 
FROM Customers c 
WHERE NOT EXISTS (
  SELECT *
  FROM Orders o 
  JOIN [Order Details] od ON od.OrderId = O.OrderID
  JOIN Products p ON p.ProductID = od.ProductID 
  WHERE p.ProductName = 'Chocolade' AND
    o.CustomerID = c.CustomerID
  )

## Zadanie 11

_Znalezienie wszystkich klientów, którzy kiedykolwiek zamówili
Scottish Longbreads_

In [ ]:
SELECT * 
FROM Customers c 
WHERE EXISTS (
  SELECT * 
  FROM Orders o 
  JOIN [Order Details] od ON od.OrderID = o.OrderID
  JOIN Products p ON p.ProductID = od.ProductID 
  WHERE p.ProductName = 'ScottISh LONgbreads' AND o.CustomerID = c.CustomerID
  )

## Zadanie 12

_Znalezienie zamówień, które zawierają Scottish Longbreads, 
ale nie zawierają Chocolade_

In [ ]:
SELECT * 
FROM Orders o 
WHERE EXISTS (
  SELECT * 
  FROM [Order Details] od 
  JOIN Products p ON p.ProductID = od.ProductID 
  WHERE ProductName = 'ScottISh LONgbreads' AND od.OrderID = o.OrderID
  )
AND NOT EXISTS (
  SELECT * 
  FROM [Order Details] od 
  JOIN Products p ON p.ProductID = od.ProductID 
  WHERE ProductName = 'Chocolade' AND od.OrderID = o.OrderID
  )

## Zadanie 13

_Znalezienie danych wszystkich pracowników, którzy 
obsługiwali zamówienia klienta ALFKI_

_Oczekiwany format wyniku: Imię i nazwisko pracownika_

In [ ]:
SELECT e.FirstName, e.LAStName 
FROM Employees e 
WHERE EXISTS (
  SELECT * 
  FROM Orders o
  WHERE o.CustomerID = 'ALFKI' AND e.EmployeeID = o.EmployeeID
)

## Zadanie 14

Przygotowanie raportu zawierającego następujące dane: imię 
pracownika, nazwisko pracownika, data zamówienia, 
informacja, czy zamówienie zawierało Chocolate (0/1). 

_W raporcie należy uwzględnić każdego pracownika_



Klauzula CASE WHEN umożliwia nam zwrócenie różnych wartości w zależności od wyniku warunku logicznego zapisanego po WHEN. W ogólności klazul WHEN może być wiele

```
CASE
    WHEN bool_1 THEN val_1 
    WHEN bool_2 THEN val_2
    WHEN ...
    ELSE val_n
END AS new_column
```

In [ ]:
SELECT 
  e.FirstName, 
  e.LAStName, 
  o.OrderDate,
  (CASE WHEN od.OrderID IS NULL THEN 0 ELSE 1 END) AS 'Status' 
FROM Employees e
LEFT JOIN Orders o ON o.EmployeeID = e.EmployeeID
LEFT JOIN [Order Details] od ON o.OrderID = od.OrderID AND od.ProductID = (
  SELECT ProductID 
  FROM Products 
  WHERE ProductName = 'Chocolade'
  )

## Zadanie 15

_Przygotowanie raportu zawierającego następujące dane: nazwa produktu, 
kraj dostawy, numer zamówienia, rok zamówienia, miesiąc zamówienia, 
data zamówienia posortowanego w malejącej kolejności dat zamówienia. W 
raporcie należy uwzględnić tylko zamówienia złożone przez klientów z 
Niemiec i produkty o nazwach rozpoczynających się na literę z przedziału 
[c-s]_


__LIKE__ umożliwia sprawdzenie przynależności słowa występującego w kolumnie do języka generowanego przez podane wyrażenie regularne.

Najczęściej stosowane operatory:

- __%__ - dowolny ciąg znaków
- __[c-s]__ - jeden znak z przedziału od c do s

In [ ]:
SELECT 
	p.ProductName,
	O.ShipCountry,
	O.OrderID,
	YEAR(O.OrderDate) as year,
	MONTH(O.OrderDate) as month,
	O.OrderDate
FROM Customers C
join Orders O on O.CustomerID = C.CustomerID
join [Order Details] od on od.OrderID = O.OrderID
join ProDucts p on p.ProductID = od.ProductID
where c.Country = 'Germany' and p.ProductName like '[c-s]%'
order by o.OrderDate desc
